# Dependences

In [7]:
include("../../src/struct_data.jl")

FusionAggregate (generic function with 1 method)

# Input Data

# Model Parameter

In [2]:
@make_struct_func Cubic
@time Model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 10000.0,
        dt    = 0.5,
        nₖₙₙ  = 50,
        nₛₐᵥₑ = 100
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input = "../../data/init/Sphere"
    )
)
dump(Model)

  0.000002 seconds (3 allocations: 112 bytes)
ModelSet
  Time: TimeModel
    tₛᵢₘ: Float64 10000.0
    dt: Float64 0.5
    nₖₙₙ: Int64 50
    nₛₐᵥₑ: Int64 100
  Input: InputModel
    outer_ratio: Float64 0.8
    path_input: String "../../data/init/Sphere"


# Aggregate

## Fusion Aggs

### First

In [3]:
AGG = Nothing
@time AGG = AggType(
    "HEK", 
    InteractionPar(
        Cubic(1.0,2.0,3.0), 
        ContractilePar(0.01)
    ),
    Float64.(readdlm("../../data/init/Sphere/15.0.xyz")[3:end,2:end]) |> cu
)
println("===================================== One Aggregate =====================================")
dump(AGG)

  1.965642 seconds (1.82 M allocations: 105.449 MiB, 1.25% gc time, 20.70% compilation time)
===================================== One Aggregate =====================================
AggType
  Name: String "HEK"
  Interaction: InteractionPar
    Force: Cubic{Float64}
      μ₁: Float64 1.0
      rₘᵢₙ: Float64 2.0
      rₘₐₓ: Float64 3.0
    Contractile: ContractilePar
      fₚ: Float64 0.01
  Radius: Float32 15.27f0
  Position: Array{Float32}((2504, 3)) Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88]
  Type: CuArray{Float32, 2, CUDA.Mem.DeviceBuffer} <: GPUArraysCore.AbstractGPUMatrix{Float32}
    storage::Union{Nothing, CUDA.ArrayStorage{CUDA.Mem.DeviceBuffer}}
    maxsize::Int64
    offset::Int64
    dims::Tuple{Int64, Int64}


In [9]:
println("=================================== Fusion Aggregate ====================================")
FusionAGG = Nothing
@time FusionAGG = FusionAggregate([AGG], Model)
show_aggregates(FusionAGG)

# TEXT = hcat(FusionAGG.Geometry.outline,Matrix(FusionAGG.Position))
# open("fusion_test.xyz", "w") do f
#     write(f, "$(size(FusionAGG.Position, 1))\n")
#     write(f, "t=$(0)\n")
#     writedlm(f,TEXT, ' ')
# end

=================================== Fusion Aggregate ====================================


LoadError: UndefVarError: ExtractData not defined